# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import csv to data frame
cities_df = pd.read_csv ('output_data/cities_data.csv')

#display data frame
cities_df.head()



,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,42.46,87,75,10.36,AR,1632186512
1,Puerto Ayora,-0.7393,-90.3518,68.09,83,40,10.40,EC,1632186596
2,Tuktoyaktuk,69.4541,-133.0374,39.20,87,90,16.11,CA,1632186708
3,East London,-33.0153,27.9116,57.63,55,9,25.72,ZA,1632186708
4,Poum,-20.2333,164.0167,72.70,64,16,20.18,NC,1632186709


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#set vaiables
locations = cities_df[['Latitude', 'Longitude']]
humidity = cities_df['Humidity']

humidity

0      87
1      83
2      87
3      55
4      64
       ..
603    78
604    94
605    23
606    84
607    84
Name: Humidity, Length: 608, dtype: int64

In [4]:
#create fig
fig = gmaps.figure()

#create heat map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, point_radius=5)

#add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#create ideal weather data frame based on conditions
ideal_weather_df = cities_df.loc[(cities_df["Humidity"] < 40) & (cities_df["Cloudiness"] <1) & (cities_df["Max Temp"] <= 80) & (cities_df["Max Temp"] >= 60)]

#drop null
ideal_weather_df = ideal_weather_df.dropna()

#reset index
ideal_weather_df = ideal_weather_df.reset_index(drop=True)

#view data frame
ideal_weather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Carnarvon,-24.8667,113.6333,75.96,33,0,20.85,AU,1632186717
1,Mizdah,31.4451,12.9801,78.35,22,0,6.89,LY,1632186724
2,Baharly,38.4362,57.4316,67.23,29,0,6.26,TM,1632186786
3,Buraidah,26.3260,43.9750,78.60,18,0,5.14,SA,1632186788
4,Abhā,18.2164,42.5053,62.85,24,0,5.75,SA,1632186826
5,Riyadh,24.6877,46.7219,77.14,18,0,5.57,SA,1632186828
6,Kalabo,-14.9700,22.6814,65.32,22,0,2.33,ZM,1632186840
7,Geraldton,-28.7667,114.6000,70.21,30,0,21.85,AU,1632186643
8,Birjand,32.8663,59.2211,66.56,23,0,6.91,IR,1632186865
9,Kalat,29.0225,66.5916,62.69,18,0,1.03,PK,1632186922


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create hotel df
hotel_df = ideal_weather_df

#add hotel column
hotel_df['Hotel Name'] = ""

#display data frame
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Carnarvon,-24.8667,113.6333,75.96,33,0,20.85,AU,1632186717,
1,Mizdah,31.4451,12.9801,78.35,22,0,6.89,LY,1632186724,
2,Baharly,38.4362,57.4316,67.23,29,0,6.26,TM,1632186786,
3,Buraidah,26.3260,43.9750,78.60,18,0,5.14,SA,1632186788,
4,Abhā,18.2164,42.5053,62.85,24,0,5.75,SA,1632186826,
5,Riyadh,24.6877,46.7219,77.14,18,0,5.57,SA,1632186828,
6,Kalabo,-14.9700,22.6814,65.32,22,0,2.33,ZM,1632186840,
7,Geraldton,-28.7667,114.6000,70.21,30,0,21.85,AU,1632186643,
8,Birjand,32.8663,59.2211,66.56,23,0,6.91,IR,1632186865,
9,Kalat,29.0225,66.5916,62.69,18,0,1.03,PK,1632186922,


In [7]:
#set up url
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

for index, row in hotel_df.iterrows():
        
        #set search params for each row
        params = {'location' : str(row["Latitude"]) + "," +str(row["Longitude"]), 'radius' : 5000, 'types' : 'Hotel', 'keyword' : 'Hotel', 'key' : g_key}
        
        #build each query
        response = requests.get(url, params=params).json()
        
        #add first hotel name from result or notify no hearby hotels
        try:
            hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
            
        except:
            print ('No nearby hotels found. Checking next location')

#display hotel data frame
hotel_df

No nearby hotels found. Checking next location


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Carnarvon,-24.8667,113.6333,75.96,33,0,20.85,AU,1632186717,Hospitality Carnarvon
1,Mizdah,31.4451,12.9801,78.35,22,0,6.89,LY,1632186724,فندق مزدة الكبير
2,Baharly,38.4362,57.4316,67.23,29,0,6.26,TM,1632186786,
3,Buraidah,26.3260,43.9750,78.60,18,0,5.14,SA,1632186788,Best Western Plus Buraidah
4,Abhā,18.2164,42.5053,62.85,24,0,5.75,SA,1632186826,Beautat Hotel
5,Riyadh,24.6877,46.7219,77.14,18,0,5.57,SA,1632186828,"Radisson Blu Hotel, Riyadh"
6,Kalabo,-14.9700,22.6814,65.32,22,0,2.33,ZM,1632186840,Sikakubete Guest House Kalabo Town Council
7,Geraldton,-28.7667,114.6000,70.21,30,0,21.85,AU,1632186643,Ocean Centre Hotel
8,Birjand,32.8663,59.2211,66.56,23,0,6.91,IR,1632186865,هتل پرويز
9,Kalat,29.0225,66.5916,62.69,18,0,1.03,PK,1632186922,Balochistan Hotel


In [8]:
#remove any rows that do not have hotels
hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]

#reset index
hotel_df = hotel_df.reset_index(drop=True)

#display data frame
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Carnarvon,-24.8667,113.6333,75.96,33,0,20.85,AU,1632186717,Hospitality Carnarvon
1,Mizdah,31.4451,12.9801,78.35,22,0,6.89,LY,1632186724,فندق مزدة الكبير
2,Buraidah,26.3260,43.9750,78.60,18,0,5.14,SA,1632186788,Best Western Plus Buraidah
3,Abhā,18.2164,42.5053,62.85,24,0,5.75,SA,1632186826,Beautat Hotel
4,Riyadh,24.6877,46.7219,77.14,18,0,5.57,SA,1632186828,"Radisson Blu Hotel, Riyadh"
5,Kalabo,-14.9700,22.6814,65.32,22,0,2.33,ZM,1632186840,Sikakubete Guest House Kalabo Town Council
6,Geraldton,-28.7667,114.6000,70.21,30,0,21.85,AU,1632186643,Ocean Centre Hotel
7,Birjand,32.8663,59.2211,66.56,23,0,6.91,IR,1632186865,هتل پرويز
8,Kalat,29.0225,66.5916,62.69,18,0,1.03,PK,1632186922,Balochistan Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))